In [71]:
import math
from typing import List
import sys
import os
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split
from copy import deepcopy
import torch.optim as optim
from transformers import AutoTokenizer, AutoModel
import optuna
from optuna.trial import TrialState
from sklearn.model_selection import KFold
import statistics
import time

In [72]:
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

In [73]:
from utils import calculate_metric

In [74]:
INPUT = "../data/data_removing_na.xlsx"

MODEL_PATH = 'best_resnet_5_fold_emb_model.pth'
BATCH_SIZE = 64
BATCH_NORM = True

DEVICE = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
DEVICE_LIST = [2, 1, 3]

MODEL = 'ProsusAI/finbert' #'AdaptLLM/finance-LLM'

In [75]:
DEVICE

device(type='cuda', index=2)

In [215]:
df = pd.read_excel(INPUT)

In [216]:
df.head()

,rr1_30,currency,seniorioty_adj,coupon rate,domicile_country,exchange_country,Industry_sector,Industry_group,Industry_subgroup,event_type,...,PD_55_pd,PD_56_pd,PD_57_pd,PD_58_pd,PD_59_pd,PD_60_pd,DTD,NI_Over_TA,Size,defaulted_in_last_6_months
0,0.259908,USD,Senior Subordinated Unsecured,9.000,United States,United States,Consumer Discretionary,Retail & Whsle - Discretionary,E-Commerce Discretionary,Bankruptcy Filing,...,0.396731,0.397453,0.398148,0.398819,0.399467,0.400092,-0.732815,-0.007137,-0.852484,False
1,0.032729,USD,Senior Subordinated Unsecured,5.750,United States,United States,Health Care,Health Care,Health Care Facilities & Svcs,Default Corp Action,...,0.957454,0.957467,0.957480,0.957492,0.957503,0.957514,-1.666262,-0.000286,-1.186347,False
2,0.972400,USD,Unsecured,5.675,South Korea,South Korea,Consumer Discretionary,Retail & Whsle - Discretionary,Wholesale - Discretionary,Default Corp Action,...,0.568169,0.568693,0.569197,0.569682,0.570150,0.570600,-1.853366,0.000191,1.053677,False
3,1.047416,CHF,Unsecured,0.125,South Korea,South Korea,Consumer Discretionary,Retail & Whsle - Discretionary,Wholesale - Discretionary,Default Corp Action,...,0.568169,0.568693,0.569197,0.569682,0.570150,0.570600,-1.853366,0.000191,1.053677,False
4,0.848872,JPY,Unsecured,1.750,Japan,Japan,Industrials,Industrial Products,Electrical Equipment,Bankruptcy Filing,...,0.130285,0.130688,0.131081,0.131465,0.131840,0.132206,-0.768857,-0.028058,-1.946507,False


In [217]:
df.shape

(1725, 165)

In [218]:
feature_list = df.columns
category_feature_key = ['currency', 'seniorioty', 'domicile_country', 'exchange_country', 'Industry_sector', 'Industry_group', 'Industry_subgroup', 'event_type',
                        'event_type_subcategory_sum', 'defaulted_in_last_6_months', 'defaulted_in_last_5_years']
category_features = [i for i in feature_list if any(sub in i for sub in category_feature_key)]
non_category_features = [i for i in feature_list if i not in category_features]

print(len(category_features))
print(len(non_category_features))

11
154


In [219]:
non_category_features.remove('rr1_30')

In [220]:
non_category_features

['coupon rate',
 'SP500 MD',
 'Average daily 1-year SP500 return',
 'Ratio to MA',
 'US Corporate Bond Yield Spread',
 'US Corporate Bond Yield Spread(3-5 year)',
 'US Corporate Bond Yield Spread(5-7 year)',
 'US Corporate Bond Yield Spread(7-10 year)',
 'US Corporate Bond Yield Spread(10+ year)',
 'US Generic Govt 3 Month Yield',
 'US Generic Govt 6 Month Yield',
 'US Generic Govt 12 Month Yield',
 'US Generic Govt 2 Year Yield',
 'US Generic Govt 3 Year Yield',
 'US Generic Govt 5 Year Yield',
 'US Generic Govt 7 Year Yield',
 'US Generic Govt 10 Year Yield',
 'marketcap',
 'sector_domicile_dtd',
 'sector_exchange_dtd',
 'sector_dtd',
 'PD_1_domicile_sector',
 'PD_3_domicile_sector',
 'PD_12_domicile_sector',
 'PD_1_domicile_subsec',
 'PD_3_domicile_subsec',
 'PD_12_domicile_subsec',
 'PD_1_exch_sector',
 'PD_3_exch_sector',
 'PD_12_exch_sector',
 'PD_1_exch_subsector',
 'PD_3_exch_subsector',
 'PD_12_exch_subsector',
 'PD_1_global_sector',
 'PD_3_global_sector',
 'PD_12_global_secto

In [221]:
category_features

['currency',
 'seniorioty_adj',
 'domicile_country',
 'exchange_country',
 'Industry_sector',
 'Industry_group',
 'Industry_subgroup',
 'event_type',
 'event_type_subcategory_sum',
 'defaulted_in_last_5_years',
 'defaulted_in_last_6_months']

In [222]:
for i in category_features:
    if i == "seniorioty_adj":
        df[i] = df[i] + " Bond"
    elif i == "domicile_country":
        df[i] = "domicile country: " + df[i]
    elif i == "exchange_country":
        df[i] = "exchange_country: " + df[i]
    elif i == "event_type":
        df[i] = "default reason: " + df[i]
    elif i == "defaulted_in_last_6_months":
        df[i] = df[i].apply(lambda x: "The bond has defaulted in the last 6 months" if x == True else "The bond has not defaulted in the last 6 months")
    elif i == "defaulted_in_last_5_years":
        df[i] = df[i].apply(lambda x: "The bond has defaulted in the last 5 years" if x == True else "The bond has not defaulted in the last 5 years")

In [223]:
df[category_features]

,currency,seniorioty_adj,domicile_country,exchange_country,Industry_sector,Industry_group,Industry_subgroup,event_type,event_type_subcategory_sum,defaulted_in_last_5_years,defaulted_in_last_6_months
0,USD,Senior Subordinated Unsecured Bond,domicile country: United States,exchange_country: United States,Consumer Discretionary,Retail & Whsle - Discretionary,E-Commerce Discretionary,default reason: Bankruptcy Filing,Debt Restructuring,The bond has not defaulted in the last 5 years,The bond has not defaulted in the last 6 months
1,USD,Senior Subordinated Unsecured Bond,domicile country: United States,exchange_country: United States,Health Care,Health Care,Health Care Facilities & Svcs,default reason: Default Corp Action,Missing Coupon payment only,The bond has not defaulted in the last 5 years,The bond has not defaulted in the last 6 months
2,USD,Unsecured Bond,domicile country: South Korea,exchange_country: South Korea,Consumer Discretionary,Retail & Whsle - Discretionary,Wholesale - Discretionary,default reason: Default Corp Action,Missing Interest payment,The bond has not defaulted in the last 5 years,The bond has not defaulted in the last 6 months
3,CHF,Unsecured Bond,domicile country: South Korea,exchange_country: South Korea,Consumer Discretionary,Retail & Whsle - Discretionary,Wholesale - Discretionary,default reason: Default Corp Action,Missing Interest payment,The bond has not defaulted in the last 5 years,The bond has not defaulted in the last 6 months
4,JPY,Unsecured Bond,domicile country: Japan,exchange_country: Japan,Industrials,Industrial Products,Electrical Equipment,default reason: Bankruptcy Filing,Rehabilitation,The bond has not defaulted in the last 5 years,The bond has not defaulted in the last 6 months
...,...,...,...,...,...,...,...,...,...,...,...
1720,MYR,Senior Secured Bond,domicile country: Malaysia,exchange_country: Malaysia,Consumer Discretionary,Consumer Discretionary Products,Automotive,default reason: Bankruptcy Filing,Others,The bond has not defaulted in the last 5 years,The bond has not defaulted in the last 6 months
1721,MYR,Senior Secured Bond,domicile country: Malaysia,exchange_country: Malaysia,Consumer Discretionary,Consumer Discretionary Products,Automotive,default reason: Bankruptcy Filing,Others,The bond has not defaulted in the last 5 years,The bond has not defaulted in the last 6 months
1722,MYR,Senior Secured Bond,domicile country: Malaysia,exchange_country: Malaysia,Consumer Discretionary,Consumer Discretionary Products,Automotive,default reason: Bankruptcy Filing,Others,The bond has not defaulted in the last 5 years,The bond has not defaulted in the last 6 months
1723,MYR,Senior Secured Bond,domicile country: Malaysia,exchange_country: Malaysia,Consumer Discretionary,Consumer Discretionary Products,Automotive,default reason: Bankruptcy Filing,Others,The bond has not defaulted in the last 5 years,The bond has not defaulted in the last 6 months


In [224]:
labels = df["rr1_30"]
features = df.drop(columns=["rr1_30"])

In [225]:
features.shape

(1725, 164)

In [226]:
# split data into training and val set
test_size = 0.25
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=test_size, random_state=42)

print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Val Features Shape:', test_features.shape)
print('Val Labels Shape:', test_labels.shape)

Training Features Shape: (1293, 164)
Training Labels Shape: (1293,)
Val Features Shape: (432, 164)
Val Labels Shape: (432,)


In [228]:
train_features.head()

,currency,seniorioty_adj,coupon rate,domicile_country,exchange_country,Industry_sector,Industry_group,Industry_subgroup,event_type,event_type_subcategory_sum,...,PD_55_pd,PD_56_pd,PD_57_pd,PD_58_pd,PD_59_pd,PD_60_pd,DTD,NI_Over_TA,Size,defaulted_in_last_6_months
755,USD,Senior Unsecured Bond,7.500,domicile country: United States,exchange_country: United States,Energy,Oil & Gas,Oil & Gas Producers,default reason: Default Corp Action,Missing Coupon payment only,...,0.691011,0.691382,0.691738,0.692080,0.692408,0.692725,-1.084433,-0.052027,-2.074964,The bond has not defaulted in the last 6 months
588,USD,Unsecured Bond,6.000,domicile country: Indonesia,exchange_country: Indonesia,Materials,Materials,Metals & Mining,default reason: Default Corp Action,Debt Restructuring,...,0.183801,0.184996,0.186167,0.187313,0.188437,0.189539,-0.540409,0.017209,0.864692,The bond has not defaulted in the last 6 months
585,USD,Senior Secured Bond,11.000,domicile country: United States,exchange_country: United States,Materials,Materials,Metals & Mining,default reason: Default Corp Action,Debt Restructuring,...,0.278774,0.280216,0.281615,0.282972,0.284290,0.285571,0.754647,-0.010395,-0.342209,The bond has not defaulted in the last 6 months
1329,USD,Senior Secured Bond,9.125,domicile country: United States,exchange_country: United States,Energy,Oil & Gas,Oil & Gas Services & Equip,default reason: Default Corp Action,Missing Coupon & principal payment,...,0.190375,0.191471,0.192544,0.193594,0.194622,0.195628,-0.242080,-0.022618,-2.808528,The bond has not defaulted in the last 6 months
973,USD,Senior Secured Bond,9.250,domicile country: United States,exchange_country: United States,Energy,Oil & Gas,Oil & Gas Producers,default reason: Default Corp Action,Others,...,0.149501,0.150930,0.152327,0.153692,0.155027,0.156332,1.138686,0.000033,-0.085154,The bond has not defaulted in the last 6 months


In [229]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModel.from_pretrained(MODEL)
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [230]:
# empty cache first
torch.cuda.empty_cache()

In [231]:
model = nn.DataParallel(model, device_ids = DEVICE_LIST)
model.to(DEVICE)

DataParallel(
  (module): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [232]:
# Function to create batches
def create_batches(inputs, batch_size):
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    
    for i in range(0, len(input_ids), batch_size):
        input_batch = {
            "input_ids": input_ids[i:i + batch_size].to(DEVICE),
            "attention_mask": attention_mask[i:i + batch_size].to(DEVICE),
        }
        yield input_batch

In [233]:
# Convert the non-categorical columns to tensors
train_features_np = train_features[non_category_features].values.astype('float32')
train_features_tensors = torch.tensor(train_features_np, dtype=torch.float32)

test_features_np = test_features[non_category_features].values.astype('float32')
test_features_tensors = torch.tensor(test_features_np, dtype=torch.float32)

In [234]:
train_features_tensors.shape

torch.Size([1293, 153])

In [235]:
batch_size = 2  # Adjust batch size based on your GPU memory

for i in category_features:
    # empty cache first
    torch.cuda.empty_cache()
    
    uniqueCats = list(set(df[i]))
    
    inputs = tokenizer(uniqueCats, return_tensors="pt", padding=True, truncation=True)
    inputs = {key: value.to(DEVICE) for key, value in inputs.items()}
    
    # create input batches
    batches = list(create_batches(inputs, batch_size))
    
    # Get the embeddings
    all_embeddings = []

    for batch in batches:
        with torch.no_grad():
            outputs = model(**batch)
            embeddings = outputs.last_hidden_state.mean(dim=1)

            # The embeddings are in the shape [batch_size, sequence_length, hidden_size]
            all_embeddings.append(embeddings)
            
    
    cpu_embeddings = torch.cat(all_embeddings, dim=0).to("cpu")
    catDict = {key: value for key, value in zip(uniqueCats, cpu_embeddings)}
    
    #### map to train_features
    train_features[i] = train_features[i].map(catDict)

    # Convert the 'category_tensor' column to a list of tensors and stack them
    train_category_tensors = torch.stack(train_features[i].tolist())

    # Concatenate the tensors along the second dimension (columns)
    train_features_tensors = torch.cat((train_features_tensors, train_category_tensors), dim=1)
    
    print(f"Shape of train_category_tensors: {train_category_tensors.shape} for feature {i}")
    print(f"Shape of train_feature_tensors: {train_features_tensors.shape}")
    
    #### map to test_features
    test_features[i] = test_features[i].map(catDict)

    # Convert the 'category_tensor' column to a list of tensors and stack them
    test_category_tensors = torch.stack(test_features[i].tolist())

    # Concatenate the tensors along the second dimension (columns)
    test_features_tensors = torch.cat((test_features_tensors, test_category_tensors), dim=1)
    
    print(f"Shape of test_category_tensors: {test_category_tensors.shape} for feature {i}")
    print(f"Shape of test_feature_tensors: {test_features_tensors.shape}")

Shape of train_category_tensors: torch.Size([1293, 768]) for feature currency
Shape of train_feature_tensors: torch.Size([1293, 921])
Shape of test_category_tensors: torch.Size([432, 768]) for feature currency
Shape of test_feature_tensors: torch.Size([432, 921])
Shape of train_category_tensors: torch.Size([1293, 768]) for feature seniorioty_adj
Shape of train_feature_tensors: torch.Size([1293, 1689])
Shape of test_category_tensors: torch.Size([432, 768]) for feature seniorioty_adj
Shape of test_feature_tensors: torch.Size([432, 1689])
Shape of train_category_tensors: torch.Size([1293, 768]) for feature domicile_country
Shape of train_feature_tensors: torch.Size([1293, 2457])
Shape of test_category_tensors: torch.Size([432, 768]) for feature domicile_country
Shape of test_feature_tensors: torch.Size([432, 2457])
Shape of train_category_tensors: torch.Size([1293, 768]) for feature exchange_country
Shape of train_feature_tensors: torch.Size([1293, 3225])
Shape of test_category_tensors: t

In [103]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.labels[idx]
        return torch.tensor(feature, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

TUNING PARAMETERS

In [121]:
# Define the ResNetBlock
class ResNetBlock(nn.Module):
    def __init__(self, in_features: int, out_features: int, dropout=0.5, batch_norm = True, activation_name='ReLU', negative_slope=0.01):
        super(ResNetBlock, self).__init__()
        if batch_norm:
            self.batch_norm = nn.BatchNorm1d(in_features)
        else:
            self.batch_norm = nn.Identity()

        self.linear = nn.Linear(in_features, out_features)
        self.relu = getattr(nn, activation_name)()
        
        if self.relu == nn.LeakyReLU():
            self.relu = self.relu(negative_slope)
            
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(out_features, in_features)

    def forward(self, x):
        resisual = x
        out = self.batch_norm(resisual)
        out = self.linear(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.linear2(out)
        out = self.dropout(out)
        out += resisual # Add the input tensor to the output
        return out

# Define the Prediction layer
class Prediction(nn.Module):
    def __init__(self, in_features: int, out_features: int, batchNorm = True):
        super(Prediction, self).__init__()
        if batchNorm:
            self.batch_norm = nn.BatchNorm1d(in_features)
        else:
            self.batch_norm = nn.Identity()

        self.relu = nn.ReLU()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x):
        x = self.batch_norm(x)
        x = self.relu(x)
        x = self.linear(x)
        return x

# Define the ResNet model
class ResNet(nn.Module):
    def __init__(self, input_dim: int, dropout: float, num_blocks: int, out_features_list: List[int], batch_norm = True, activation_name='ReLU', negative_slope=0.01):
        super(ResNet, self).__init__()
        self.layers = nn.ModuleList()
        
        # First linear layer to convert input_dim to the first out_features
        self.layers.append(nn.Linear(input_dim, out_features_list[0]))
        
        # Add the ResNet blocks
        for i in range(num_blocks):
            if i > 0:
                self.layers.append(ResNetBlock(out_features_list[0], out_features_list[i], dropout, batch_norm = batch_norm, activation_name=activation_name, negative_slope=negative_slope))
        
        # Add the prediction layer
        self.prediction = Prediction(out_features_list[0], 1, batchNorm = batch_norm)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = self.prediction(x)
        return x

In [118]:
# define loss function
criterion = nn.MSELoss()

In [110]:
EPOCHS = 300

In [111]:
# set random seed
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [238]:
train_features_np = train_features_tensors.numpy()
test_features_np = test_features_tensors.numpy()

In [114]:
scaler = StandardScaler()

In [147]:
def objective(trial):    
    # Generate the model.
    num_blocks = trial.suggest_int('num_blocks', 1, 5)
    
    out_features_list = []
    for i in range(num_blocks):
        out_features = trial.suggest_int(f'out_features_{i}', 10, 128)
        out_features_list.append(out_features)

    p = trial.suggest_float("dropout", 0, 0.5)
    # activation_name = trial.suggest_categorical(f'activation_{i}', ['ReLU', 'Tanh', 'LeakyReLU'])
    # negative_slope = 0.01
    # if activation_name == 'LeakyReLU':
    #     negative_slope = trial.suggest_float(f"negative_slope_{i}", 0.01, 1)
        

    # model = ResNet(input_dim=train_features.shape[1], num_blocks=num_blocks, dropout=p, out_features_list=out_features_list, batch_norm=BATCH_NORM).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-9, 1e-1, log=True)


    # Define cross-validation setup
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    val_losses = []

    for train_idx, val_idx in kf.split(train_features_np):
        # Create training and validation datasets for the current fold
        X_train_fold, X_val_fold = train_features_np[train_idx], train_features_np[val_idx]
        y_train_fold, y_val_fold = train_labels.values.reshape(-1, 1)[train_idx], train_labels.values.reshape(-1, 1)[val_idx]
        
        # scaling features
        X_train_fold[:, :len(non_category_features)] = scaler.fit_transform(X_train_fold[:, :len(non_category_features)])
        X_val_fold[:, :len(non_category_features)] = scaler.transform(X_val_fold[:, :len(non_category_features)])
            
        # Initialize the model for this fold
        model = ResNet(input_dim=X_train_fold.shape[1], num_blocks=num_blocks, dropout=p, out_features_list=out_features_list, batch_norm=BATCH_NORM) #activation_name=activation_name, negative_slope=negative_slope)
        model = nn.DataParallel(model, device_ids = DEVICE_LIST)
        model.to(DEVICE)
        
        # define optimizer
        if optimizer_name == "Adam":
         optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr, weight_decay=weight_decay)
        else:
            momentum = trial.suggest_float("momentum", 1e-9, 0.95, log=True)
            optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr, weight_decay=weight_decay, momentum=momentum)
        
        # Define the loss function and optimizer
        criterion = nn.MSELoss()
        
        # Prepare DataLoader for training
        train_dataset = CustomDataset(X_train_fold, y_train_fold)
        val_dataset = CustomDataset(X_val_fold, y_val_fold)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
        
        # print(y_val_fold.shape)
            
        # Training of the model.
        model.train()
        for epoch in range(EPOCHS):
            for batch_idx, (data, target) in enumerate(train_loader):
                data, target = data.to(DEVICE), target.to(DEVICE)

                optimizer.zero_grad()
                output = model(data)
                # print('shape', output.shape, target.shape)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()

        # Validation of the model.
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(val_loader):
                data, target = data.to(DEVICE), target.to(DEVICE)
                output = model(data)
                val_loss = criterion(output, target).item()
                val_losses.append(val_loss**0.5) #rmse

        trial.report(val_loss, epoch)

    # Return the average validation loss across all folds
    return np.mean(val_losses)

In [148]:
# 3. Create a study object and optimize the objective function.
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100, timeout=600)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


[I 2024-08-30 03:38:02,172] A new study created in memory with name: no-name-6a6083d6-91c4-4ad8-87e8-8949a2260409
/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 299 is already reported.
  warnings.warn(
/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 299 is already reported.
  warnings.warn(
/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 299 is already reported.
  warnings.warn(
/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 299 is already reported.
  warnings.warn(
[I 2024-08-30 03:43:31,739] Trial 0 finished with value: 0.2506820472087633 and parame

Study statistics: 
  Number of finished trials:  2
  Number of pruned trials:  0
  Number of complete trials:  2
Best trial:
  Value:  0.2506820472087633
  Params: 
    num_blocks: 4
    out_features_0: 58
    out_features_1: 11
    out_features_2: 123
    out_features_3: 86
    dropout: 0.38721048119932927
    optimizer: Adam
    lr: 8.69746101373016e-05
    weight_decay: 0.0008572312583192992


In [149]:
trial.params

{'num_blocks': 4,
 'out_features_0': 58,
 'out_features_1': 11,
 'out_features_2': 123,
 'out_features_3': 86,
 'dropout': 0.38721048119932927,
 'optimizer': 'Adam',
 'lr': 8.69746101373016e-05,
 'weight_decay': 0.0008572312583192992}

In [150]:
MODEL_CONFIG = {"out_features_list": [], "dropout": 0, "optimizer": {}, "num_blocks": 0}

for key, value in trial.params.items():
    if "out_features" in key:
        MODEL_CONFIG["out_features_list"].append(value)
    elif "dropout" in key:
        MODEL_CONFIG["dropout"] = value
    elif "negative_slope" in key:
        MODEL_CONFIG["negative_slope"] = value
    elif "activation" in key:
        MODEL_CONFIG["activation_name"] = value
    elif "num_blocks" in key:
        MODEL_CONFIG["num_blocks"] = value
    elif "batch_size" in key:
        BATCH_SIZE = int(value)
    else:
        MODEL_CONFIG["optimizer"][key] = value

In [151]:
MODEL_CONFIG

{'out_features_list': [58, 11, 123, 86],
 'dropout': 0.38721048119932927,
 'optimizer': {'optimizer': 'Adam',
  'lr': 8.69746101373016e-05,
  'weight_decay': 0.0008572312583192992},
 'num_blocks': 4}

RUNNING THE MODEL

In [ ]:
# empty cache first
torch.cuda.empty_cache()

In [241]:
scaler = StandardScaler()

In [242]:
# scaling features
train_features_np[:,:len(non_category_features)] = scaler.fit_transform(train_features_np[:,:len(non_category_features)])
test_features_np[:,:len(non_category_features)] = scaler.transform(test_features_np[:,:len(non_category_features)])
            
# Create dataset instances
train_dataset = CustomDataset(train_features_np, train_labels.values.reshape(-1, 1))
test_dataset = CustomDataset(test_features_np, test_labels.values.reshape(-1, 1))

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=test_features_np.shape[0], shuffle=True)

In [245]:
m_config = deepcopy(MODEL_CONFIG)
del m_config["optimizer"]

In [246]:
m_config

{'out_features_list': [58, 11, 123, 86],
 'dropout': 0.38721048119932927,
 'num_blocks': 4}

In [263]:
model = ResNet(input_dim=train_features_np.shape[1], batch_norm=BATCH_NORM, **m_config)
model

ResNet(
  (layers): ModuleList(
    (0): Linear(in_features=8601, out_features=58, bias=True)
    (1): ResNetBlock(
      (batch_norm): BatchNorm1d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (linear): Linear(in_features=58, out_features=11, bias=True)
      (relu): ReLU()
      (dropout): Dropout(p=0.38721048119932927, inplace=False)
      (linear2): Linear(in_features=11, out_features=58, bias=True)
    )
    (2): ResNetBlock(
      (batch_norm): BatchNorm1d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (linear): Linear(in_features=58, out_features=123, bias=True)
      (relu): ReLU()
      (dropout): Dropout(p=0.38721048119932927, inplace=False)
      (linear2): Linear(in_features=123, out_features=58, bias=True)
    )
    (3): ResNetBlock(
      (batch_norm): BatchNorm1d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (linear): Linear(in_features=58, out_features=86, bias=True)
      (relu): ReLU()


In [264]:
model = nn.DataParallel(model, device_ids = DEVICE_LIST)
model.to(DEVICE)

DataParallel(
  (module): ResNet(
    (layers): ModuleList(
      (0): Linear(in_features=8601, out_features=58, bias=True)
      (1): ResNetBlock(
        (batch_norm): BatchNorm1d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (linear): Linear(in_features=58, out_features=11, bias=True)
        (relu): ReLU()
        (dropout): Dropout(p=0.38721048119932927, inplace=False)
        (linear2): Linear(in_features=11, out_features=58, bias=True)
      )
      (2): ResNetBlock(
        (batch_norm): BatchNorm1d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (linear): Linear(in_features=58, out_features=123, bias=True)
        (relu): ReLU()
        (dropout): Dropout(p=0.38721048119932927, inplace=False)
        (linear2): Linear(in_features=123, out_features=58, bias=True)
      )
      (3): ResNetBlock(
        (batch_norm): BatchNorm1d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (linear): Linear(i

In [265]:
# define optimizer
optim_config = deepcopy(MODEL_CONFIG["optimizer"])
del optim_config["optimizer"]

optimizer = getattr(optim, MODEL_CONFIG["optimizer"]["optimizer"])(model.parameters(), **optim_config)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 8.69746101373016e-05
    maximize: False
    weight_decay: 0.0008572312583192992
)

In [266]:

EPOCH = 500
criterion = nn.MSELoss()
start_time = time.time()

for ep in tqdm(range(EPOCH)):

    model.train()
    running_loss = 0.0
    for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        loss = criterion(outputs, targets)
        
        loss.backward()

        optimizer.step()

        # print statistics
        running_loss += loss.item() * inputs.size(0)
        if i % 100 == 99:    # print every 100 mini-batches
            print(f'[{ep + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')


    train_loss = running_loss  / len(train_loader.dataset)
    print(f'Epoch [{ep+1}], Train Loss: {train_loss:.4f}')
    
# print out training time
elapsed_time = time.time() - start_time
print(f"Training time: {elapsed_time:.3f} seconds")

# save the best model
torch.save(model.state_dict(), MODEL_PATH)

  0%|          | 1/500 [00:00<02:10,  3.82it/s]

Epoch [1], Train Loss: 0.8174


  0%|          | 2/500 [00:00<02:27,  3.38it/s]

Epoch [2], Train Loss: 0.7034


  1%|          | 3/500 [00:00<02:26,  3.39it/s]

...

Epoch [498], Train Loss: 0.0243


100%|█████████▉| 499/500 [02:28<00:00,  3.40it/s]

Epoch [499], Train Loss: 0.0258


100%|██████████| 500/500 [02:29<00:00,  3.35it/s]

Epoch [500], Train Loss: 0.0267
Training time: 149.175 seconds


In [267]:
torch.cuda.empty_cache()

In [268]:
# Testing phase
model.eval()
test_loss = 0.0

with torch.no_grad():
    for inputs, targets in test_loader:
        # inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        
        if isinstance(model, nn.DataParallel):
            model = model.module  # Unwrap from DataParallel
        model = model.to('cpu')
        
        outputs = model(inputs)

        # save metrics
        mae, mape, rmse, rsqr = calculate_metric(outputs.numpy(), targets.numpy())
        print(f"Training average mean absolute error: {mae}")
        print(f"Training average mean absolute percentage error: {mape}")
        print(f"Training average root mean squared error: {rmse}")
        print(f"Training average R2: {rsqr}")

Training average mean absolute error: 0.1486234962940216
Training average mean absolute percentage error: 278.23894023895264
Training average root mean squared error: 0.22259254548104257
Training average R2: 0.5674374103546143


In [165]:
# let’s load back in our saved model
# model = MLP()
# model.load_state_dict(torch.load(MODEL_PATH))

RUN 5-fold

In [269]:
torch.cuda.empty_cache()

In [270]:
features = np.concatenate([train_features_np, test_features_np], axis=0)
features

array([[ 0.0765418 , -0.6170648 , -0.54855525, ..., -0.25331536,
        -0.20173293,  0.32425982],
       [-0.42837855,  0.5031553 ,  0.8686651 , ..., -0.25331536,
        -0.20173293,  0.32425982],
       [ 1.2546893 ,  0.31953788,  0.73418504, ..., -0.25331536,
        -0.20173293,  0.32425982],
       ...,
       [ 0.05971107,  1.0929172 ,  0.8371482 , ..., -0.25331536,
        -0.20173293,  0.32425982],
       [-2.2797532 , -0.02818421, -0.5013583 , ..., -0.25331536,
        -0.20173293,  0.32425982],
       [ 1.591303  , -0.22871129, -0.3548853 , ..., -0.25331536,
        -0.20173293,  0.32425982]], dtype=float32)

In [271]:
labels = pd.concat([train_labels, test_labels], axis=0, ignore_index=True)
labels

0       0.082481
1       0.378845
2       0.836149
3       0.987208
4       1.021458
          ...   
1720    0.471411
1721    0.823750
1722    0.241612
1723    0.762054
1724    0.199981
Name: rr1_30, Length: 1725, dtype: float64

In [274]:
# Define cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)
EPOCHS = 500
val_mae = []
val_mape = []
val_rmse = []
val_rsqr = []


for train_idx, val_idx in kf.split(features):
    # Create training and validation datasets for the current fold
    X_train_fold, X_val_fold = features[train_idx], features[val_idx]
    y_train_fold, y_val_fold = labels.values.reshape(-1, 1)[train_idx], labels.values.reshape(-1, 1)[val_idx]
    
    # scaling features
    X_train_fold[:,:len(non_category_features)] = scaler.fit_transform(X_train_fold[:,:len(non_category_features)])
    X_val_fold[:,:len(non_category_features)] = scaler.transform(X_val_fold[:,:len(non_category_features)])
        
    # Initialize the model for this fold
    odel = ResNet(input_dim=X_train_fold.shape[1], batch_norm=BATCH_NORM, **m_config)
    model = nn.DataParallel(model, device_ids = DEVICE_LIST)
    model.to(DEVICE)
    
    # define optimizer
    optimizer = getattr(optim, MODEL_CONFIG["optimizer"]["optimizer"])(model.parameters(), **optim_config)
    
    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    
    # Prepare DataLoader for training
    train_dataset = CustomDataset(X_train_fold, y_train_fold)
    val_dataset = CustomDataset(X_val_fold, y_val_fold)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=val_dataset.features.shape[0], shuffle=True)
        
    # Training of the model.
    model.train()
    running_loss = 0
    for epoch in range(EPOCHS):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            # print statistics
            running_loss += loss.item() * data.size(0)
            if i % 100 == 99:    # print every 100 mini-batches
                print(f'[{ep + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')


    train_loss = running_loss  / len(train_loader.dataset)
    print(f'Epoch [{ep+1}], Train Loss: {train_loss:.4f}')
            # print(f'Epoch [{ep+1}], Train Loss: {train_loss:.4f}')

    # Validation of the model.
    model.eval()
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(val_loader):
            # data, target = data.to(DEVICE), target.to(DEVICE)
            if isinstance(model, nn.DataParallel):
                model = model.module  # Unwrap from DataParallel
            model = model.to('cpu')
            outputs = model(data)
            
            # save metrics
            mae, mape, rmse, rsqr = calculate_metric(outputs.numpy(), target.numpy())
            val_mae.append(mae)
            val_mape.append(mape)
            val_rmse.append(rmse)
            val_rsqr.append(rsqr)
            

Epoch [500], Train Loss: 11.4113
Epoch [500], Train Loss: 10.1372
Epoch [500], Train Loss: 9.9146
Epoch [500], Train Loss: 9.8181
Epoch [500], Train Loss: 9.3126


In [275]:
print(f"Test average mean absolute error: {statistics.mean(val_mae)}")
print(f"Test average mean absolute percentage error: {statistics.mean(val_mape)}")
print(f"Test average root mean squared error: {statistics.mean(val_rmse)}")
print(f"Test average R2: {statistics.mean(val_rsqr)}")

Test average mean absolute error: 0.14244692027568817
Test average mean absolute percentage error: 2524.3922352790833
Test average root mean squared error: 0.21147047337581887
Test average R2: 0.5876680970191955


In [276]:
val_rmse

[0.2191690960236959,
 0.21912177033663768,
 0.21159403671562974,
 0.21641166934998218,
 0.1910557944531488]